In [7]:
import polars

from autofeat.dataset import *
from autofeat.transform import *

In [8]:
dataset = KaggleDataset(id="mahmoudemadabdallah/hr-analytics-employee-attrition-and-performance")

In [9]:
transform = (
    Cast()
    .then(Encode())
    .then(Aggregate(by={"EmployeeID"}))
    .then(Identity(), Join(on={"EmployeeID"}))
    .then(Identity(), Combine())
)

In [11]:
features = dataset.features(
    Filter(eq={"EmployeeID": "51A4-EA6E"}),
    Filter(eq={"EmployeeID": "A4A4-F49C"}),
)

In [12]:
features.collect()

count(*) from RatingLevel.csv,max(RatingID) from RatingLevel.csv,mean(RatingID) from RatingLevel.csv,median(RatingID) from RatingLevel.csv,min(RatingID) from RatingLevel.csv,std(RatingID) from RatingLevel.csv,sum(RatingID) from RatingLevel.csv,var(RatingID) from RatingLevel.csv,count(*) from PerformanceRating.csv,max(EnvironmentSatisfaction) from PerformanceRating.csv,mean(EnvironmentSatisfaction) from PerformanceRating.csv,median(EnvironmentSatisfaction) from PerformanceRating.csv,min(EnvironmentSatisfaction) from PerformanceRating.csv,std(EnvironmentSatisfaction) from PerformanceRating.csv,sum(EnvironmentSatisfaction) from PerformanceRating.csv,var(EnvironmentSatisfaction) from PerformanceRating.csv,max(ManagerRating) from PerformanceRating.csv,mean(ManagerRating) from PerformanceRating.csv,median(ManagerRating) from PerformanceRating.csv,min(ManagerRating) from PerformanceRating.csv,std(ManagerRating) from PerformanceRating.csv,sum(ManagerRating) from PerformanceRating.csv,var(ManagerRating) from PerformanceRating.csv,max(TrainingOpportunitiesTaken) from PerformanceRating.csv,mean(TrainingOpportunitiesTaken) from PerformanceRating.csv,median(TrainingOpportunitiesTaken) from PerformanceRating.csv,min(TrainingOpportunitiesTaken) from PerformanceRating.csv,std(TrainingOpportunitiesTaken) from PerformanceRating.csv,sum(TrainingOpportunitiesTaken) from PerformanceRating.csv,var(TrainingOpportunitiesTaken) from PerformanceRating.csv,max(WorkLifeBalance) from PerformanceRating.csv,mean(WorkLifeBalance) from PerformanceRating.csv,median(WorkLifeBalance) from PerformanceRating.csv,min(WorkLifeBalance) from PerformanceRating.csv,std(WorkLifeBalance) from PerformanceRating.csv,sum(WorkLifeBalance) from PerformanceRating.csv,var(WorkLifeBalance) from PerformanceRating.csv,…,sum(Age) / std(Education) from Employee.csv,mean(YearsInMostRecentRole) + mean(Education) from Employee.csv,mean(YearsInMostRecentRole) - mean(Education) from Employee.csv,mean(Education) - mean(YearsInMostRecentRole) from Employee.csv,mean(YearsInMostRecentRole) * mean(Education) from Employee.csv,mean(YearsInMostRecentRole) / mean(Education) from Employee.csv,mean(Education) / mean(YearsInMostRecentRole) from Employee.csv,mean(YearsInMostRecentRole) + median(YearsSinceLastPromotion) from Employee.csv,mean(YearsInMostRecentRole) - median(YearsSinceLastPromotion) from Employee.csv,median(YearsSinceLastPromotion) - mean(YearsInMostRecentRole) from Employee.csv,mean(YearsInMostRecentRole) * median(YearsSinceLastPromotion) from Employee.csv,mean(YearsInMostRecentRole) / median(YearsSinceLastPromotion) from Employee.csv,median(YearsSinceLastPromotion) / mean(YearsInMostRecentRole) from Employee.csv,mean(YearsInMostRecentRole) + sum(Age) from Employee.csv,mean(YearsInMostRecentRole) - sum(Age) from Employee.csv,sum(Age) - mean(YearsInMostRecentRole) from Employee.csv,mean(YearsInMostRecentRole) * sum(Age) from Employee.csv,mean(YearsInMostRecentRole) / sum(Age) from Employee.csv,sum(Age) / mean(YearsInMostRecentRole) from Employee.csv,mean(Education) + median(YearsSinceLastPromotion) from Employee.csv,mean(Education) - median(YearsSinceLastPromotion) from Employee.csv,median(YearsSinceLastPromotion) - mean(Education) from Employee.csv,mean(Education) * median(YearsSinceLastPromotion) from Employee.csv,mean(Education) / median(YearsSinceLastPromotion) from Employee.csv,median(YearsSinceLastPromotion) / mean(Education) from Employee.csv,mean(Education) + sum(Age) from Employee.csv,mean(Education) - sum(Age) from Employee.csv,sum(Age) - mean(Education) from Employee.csv,mean(Education) * sum(Age) from Employee.csv,mean(Education) / sum(Age) from Employee.csv,sum(Age) / mean(Education) from Employee.csv,median(YearsSinceLastPromotion) + sum(Age) from Employee.csv,median(YearsSinceLastPromotion) - sum(Age) from Employee.csv,sum(Age) - median(YearsSinceLastPromotion) from Employee.csv,median(YearsSinceLastPromotion) * sum(Age) from Employee.csv,median(YearsSinceLastPromotion) 